### Geo-json logic

In [1]:
import joblib
import rasterio
from rasterio.features import shapes
import numpy as np
import json
from pathlib import Path
from shapely.geometry import Polygon, mapping
import geopandas as gpd
from folium import Map, GeoJson, GeoJsonTooltip

# import geopandas as gpd
# import folium

>- Utility functions

In [2]:
# Band mapping for your UAV image
BAND_MAPPING = {
    "RED": 0,
    "NIR": 3,
    "GREEN": 1
}


# Correct band mapping
BAND_MAPPING_CORRECT_Fix4D = {
    "BLUE": 0,
    "GREEN": 1,
    "RED": 2,
    "RED_EDGE": 3,
    "NIR": 4,
}

BAND_MAPPING_CORRECT_ODM = {
    "RED": 0,
    "GREEN": 1,
    "BLUE": 2,
    "NIR": 3,
    "RED_EDGE": 4,
}

In [3]:
def calculate_ndvi(nir_band, red_band):
    """Calculates Normalized Difference Vegetation Index (NDVI)."""
    return (nir_band - red_band) / (nir_band + red_band + 1e-10)

def calculate_ndwi(nir_band, green_band):
    """Calculates Normalized Difference Water Index (NDWI)."""
    # nwir_band ---> geen_band 
    # NDWI = (green_band - nir_band) / (green_band + nir_band + 1e-10 )
    # return (nir_band - swir_band) / (nir_band + swir_band + 1e-10)
    return(green_band - nir_band) / (green_band + nir_band + 1e-10 )


def extract_features_from_patch_array(patch_array):
    """
    Extracts features from a single multispectral patch NumPy array.
    Expects patch_array shape: (num_bands, height, width)
    """
    features = []
    # Safely get band data using BAND_MAPPING
    try:
        red_band = patch_array[BAND_MAPPING["RED"]]
        nir_band = patch_array[BAND_MAPPING["NIR"]]
        green_band = patch_array[BAND_MAPPING["GREEN"]]
        
        # Calculate NDVI and NDWI
        ndvi = calculate_ndvi(nir_band, red_band)
        ndwi = calculate_ndwi(nir_band, green_band)
        
        features = [
            np.mean(ndvi), np.std(ndvi),
            np.mean(ndwi), np.std(ndwi),
            np.percentile(nir_band, 75),
            np.mean(green_band > np.quantile(green_band, 0.75))
        ]
        
        return np.array(features)
    except KeyError as e:
        raise ValueError(f"Missing required band in patch: {e}")
    
    

>- Main logic

In [4]:
# Assume these are defined elsewhere in your code
# from my_utils import extract_features_from_patch_array
# from my_constants import BAND_MAPPING
# OUTPUT_GEOJSON_PATH = "path/to/your/output.geojson"
# GROWTH_STAGES = ["stage_a", "stage_b", "stage_c"]


def process_field_for_mapping(image_path: Path, ml_model, growth_stages: list,
                              patch_size: int = 64, min_pixel_sum_threshold: int = 1000) -> Path:
    """
    Processes a multispectral GeoTIFF, extracts valid patches,
    predicts growth stages, generates a classified raster,
    and then vectorizes it into a GeoJSON file with correctly shaped polygons.
    """
    print(f">>>>>>>>>>--------- Starting processing for: {image_path.name} ---------<<<<<<<<<<", flush=True)

    with rasterio.open(image_path) as src:
        image_data = src.read()
        profile = src.profile
        transform = src.transform
        nodata_val = src.nodata

        bands, h, w = image_data.shape
        print(f":--->:) Image dimensions: {h}x{w} pixels, {bands} bands.", flush=True)

        # Create an output array for classified pixels
        # Use a data type that can hold your class labels (e.g., int8, int16)
        classified_raster = np.full((h, w), fill_value=-1, dtype=np.int16)
        
        num_patches_skipped = 0
        total_possible_patches = 0
        
        print(":--->:) Start the feeding patches to model", flush=True)
        # --- Patch-wise Prediction and Raster Filling ---
        for r_start in range(0, h, patch_size):
            for c_start in range(0, w, patch_size):
                total_possible_patches += 1
                r_end = r_start + patch_size
                c_end = c_start + patch_size
                
                # Handle partial patches at the edges
                current_h = min(patch_size, h - r_start)
                current_w = min(patch_size, w - c_start)
                
                # Skip if the patch is too small to be meaningful (optional)
                if current_h < patch_size or current_w < patch_size:
                    num_patches_skipped += 1
                    continue
                
                patch_data = image_data[:, r_start:r_end, c_start:c_end]

                # --- Filtering for "informationless" patches ---
                if nodata_val is not None and np.all(patch_data == nodata_val):
                    num_patches_skipped += 1
                    continue

                if np.sum(patch_data) < min_pixel_sum_threshold:
                    num_patches_skipped += 1
                    continue
                
                if patch_data.shape[0] < bands: # Basic check for sufficient bands
                    num_patches_skipped += 1
                    continue
                
                # Reshape patch for feature extraction
                try:
                    features = extract_features_from_patch_array(patch_data)
                    # Check for NaNs
                    if any(np.isnan(f) for f in features):
                        num_patches_skipped += 1
                        continue
                        
                    # Perform prediction
                    prediction_label = int(ml_model.predict([features])[0])
                    
                    # Fill the classified raster with the predicted label
                    classified_raster[r_start:r_end, c_start:c_end] = prediction_label
                except Exception as e:
                    print(f"Error processing patch at {r_start},{c_start}: {e}", flush=True)
                    num_patches_skipped += 1
                    continue

    print(f"===> Finished patch processing. Skipped {num_patches_skipped} patches.", flush=True)
    print("===> Now vectorizing the classified raster...", flush=True)

    # --- Vectorize the Classified Raster ---
    geojson_features = []
    
    # Set nodata value for the classified raster
    classified_raster_nodata = -1
    
    # The `shapes` function is the key to solving your problem.
    # It converts a raster with discrete values (our class labels) into polygons.
    # It automatically handles the irregular shapes by grouping adjacent pixels with the same value.
    for geom, value in shapes(
        classified_raster.astype(np.int16), 
        mask=(classified_raster != classified_raster_nodata), # Create a mask to ignore 'no-data' values
        transform=transform
    ):
        if value != -1:  # Only process the valid predicted classes
            growth_stage_name = growth_stages[int(value)]
            geojson_features.append({
                "type": "Feature",
                "geometry": geom,  # `shapes` already returns GeoJSON-compatible geometry
                "properties": {
                    "growth_stage": growth_stage_name
                }
            })
            
    if not geojson_features:
        print(":--->:( No features were vectorized. Exiting.", flush=True)
        return None
    
    # --- GeoJSON Output ---
    output_geojson_data = {
        "type": "FeatureCollection",
        "crs": {
            "type": "name",
            "properties": {"name": f"EPSG:{src.crs.to_epsg()}"}
        },
        "features": geojson_features
    }
    
    # Define the output path for the GeoJSON file
    output_dir = Path("test_new_update_mapcreation")
    output_dir.mkdir(parents=True, exist_ok=True)
    output_geojson_path = Path(f"{output_dir}/classified_output.geojson") # Or your defined path
    with open(output_geojson_path, "w") as f:
        json.dump(output_geojson_data, f, indent=2)

    print(f":---->:) GeoJSON data saved to {output_geojson_path}", flush=True)
    print("===> Processing complete. GeoJSON file with correctly shaped polygons generated.", flush=True)
    return output_geojson_path

>- call thr function

>- >- path define

In [5]:
root_dir = Path("../../")   # Adjust this path to your project root
src_dir = root_dir / "src"  # Path to your source directory
# Path to your multispectral UAV image
UAV_IMAGE_PATH = root_dir / "temp" / "odm_orthophoto" / "odm_orthophoto.tif"
if not UAV_IMAGE_PATH.exists():
    raise FileNotFoundError(f":--->:( UAV image not found at {UAV_IMAGE_PATH}")
# Path to your trained ML model
ML_MODEL_PATH = src_dir /"App" / "model" / "XGB_model_v5.joblib" 
if not ML_MODEL_PATH.exists():
    raise FileNotFoundError(f":--->:( ML model not found at {ML_MODEL_PATH}")

print(f":--->:) UAV Image Path: {UAV_IMAGE_PATH}", flush=True)
print(f":--->:) ML Model Path: {ML_MODEL_PATH}", flush=True)

:--->:) UAV Image Path: ..\..\temp\odm_orthophoto\odm_orthophoto.tif
:--->:) ML Model Path: ..\..\src\App\model\XGB_model_v5.joblib


>- >- load model

In [6]:
try:
    loaded_model = joblib.load(ML_MODEL_PATH)
except Exception as e:
    raise ValueError(f"Failed to load the model from {ML_MODEL_PATH}: {e}")

print(f":--->:) Model loaded successfully from {ML_MODEL_PATH}", flush=True)

:--->:) Model loaded successfully from ..\..\src\App\model\XGB_model_v5.joblib


In [7]:
GROWTH_STAGES = ["germination", "tillering", "grand_growth", "ripening"]
MIN_PIXEL_SUM_THRESHOLD = 5000
# Call the function to process the field and generate the GeoJSON
geojson_path = process_field_for_mapping(
    image_path=Path(UAV_IMAGE_PATH), 
    ml_model=loaded_model, 
    growth_stages=GROWTH_STAGES,
    min_pixel_sum_threshold=MIN_PIXEL_SUM_THRESHOLD
)

OUTPUT_GEOJSON_PATH = geojson_path

>>>>>>>>>>--------- Starting processing for: odm_orthophoto.tif ---------<<<<<<<<<<
:--->:) Image dimensions: 2301x1180 pixels, 6 bands.
:--->:) Start the feeding patches to model
===> Finished patch processing. Skipped 207 patches.
===> Now vectorizing the classified raster...
:---->:) GeoJSON data saved to test_new_update_mapcreation\classified_output.geojson
===> Processing complete. GeoJSON file with correctly shaped polygons generated.


### Map logic

In [8]:
# Assume `output_geojson_path` is the path returned by the function above
output_geojson_path = Path(OUTPUT_GEOJSON_PATH)

# Define a color map for the different growth stages
# You can choose your own colors here
color_map = {
    "germination": "#B3E5FC",   # Light Blue
    "tillering": "#8BC34A",     # Light Green
    "grand_growth": "#4CAF50",  # Green
    "ripening": "#FFEB3B",      # Yellow
    None: "#808080"             # Gray for skipped/unclassified patches
}

# The new style_function will use this dictionary
def style_function(feature):
    growth_stage = feature['properties'].get('growth_stage')
    return {
        'fillColor': color_map.get(growth_stage, '#808080'), # Use a fallback color for safety
        'color': 'black',
        'weight': 0.2,
        'fillOpacity': 0.7
    }

if output_geojson_path.exists():
    # Load the new GeoJSON file
    geo_json_obj = gpd.read_file(output_geojson_path)
    
    # Create a Folium map centered on the centroid of the patches
    # The to_crs is important if your GeoTIFF is not in WGS84 (EPSG:4326)
    centroid = geo_json_obj.to_crs("EPSG:4326").geometry.centroid
    
    # Handle empty GeoDataFrame
    if centroid.empty:
        print("GeoJSON file is empty. Cannot create map.")
        # Create a default map or handle the error
        m = Map(location=[0, 0], zoom_start=2) 
    else:
        m = Map(location=[centroid.y.mean(), centroid.x.mean()], zoom_start=16, tiles="OpenStreetMap")
    
    # Add GeoJSON to the map with style based on growth_stage
    GeoJson(
        geo_json_obj.to_crs("EPSG:4326"),
        style_function=style_function,
        tooltip=GeoJsonTooltip(fields=['growth_stage'])
    ).add_to(m)
    
    
    output_dir = Path("test_new_update_mapcreation/map_output")
    output_dir.mkdir(parents=True, exist_ok=True)
    # define save file path
    save_path = output_dir / "sugarcane_growth_map.html"
    
    # Save and display
    m.save(save_path)
    print(f"Interactive map saved as {save_path}")

Interactive map saved as test_new_update_mapcreation\map_output\sugarcane_growth_map.html


C:\Users\user\AppData\Local\Temp\ipykernel_8008\672086571.py:30: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = geo_json_obj.to_crs("EPSG:4326").geometry.centroid
